## Imports

In [ ]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, GlobalMaxPooling2D, AveragePooling2D, GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Parameters

In [ ]:
IMG_SHAPE = (224,224)
IMG_SHAPE_GN = (224,224,3)
steps_per_epoch = 55 #if you get warning message while fitting after changing this variable change it back to 60  samplesize//batch_size
batch_size = 32 #if you get warning message after changing this variable change it back to 32
validation_steps = 18 #if you get warning message after changing this variable change it back to 15  samplesize//batch_size
epochs = 10
df = pd.read_csv("Cats.csv")
test = pd.read_csv("test.csv")

## Utility Functions

In [ ]:
# Process image (input) for the model
def process(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = tf.keras.applications.mobilenet.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

# Read image
def get_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

## Preparing the Dataset

In [ ]:
train_datagen = ImageDataGenerator(
                                    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    validation_split=0.25,
                                    rotation_range=90,
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2
                                  )
validation_datagen = ImageDataGenerator(
                                    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
                                    validation_split=0.25,
    )
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)


In [ ]:
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory='images/',
        x_col='image',
        y_col='classname',
        target_size=IMG_SHAPE,
        batch_size=batch_size,
        class_mode="categorical",
        seed=420,
        subset='training'
)
valid_gen_flow = validation_datagen.flow_from_dataframe(
        dataframe=df,
        directory='images/',
        x_col='image',
        y_col='classname',
        target_size=IMG_SHAPE,
        batch_size=batch_size,
        class_mode="categorical",
        #seed=500,
        shuffle=False,
        subset='validation')
test_gen = test_datagen.flow_from_dataframe(
        dataframe=test,
        directory='test/test',
        x_col='image',
        y_col='classname',
        target_size=IMG_SHAPE,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False)


## Creating & Fitting the model

In [ ]:
pretrainedModel = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE_GN,
     include_top=False,
     weights='imagenet',
     pooling='avg'
)
pretrainedModel.trainable = False

inputs = pretrainedModel.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrainedModel.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(12, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['acc'])


In [ ]:
history = model.fit(
    train_gen_flow,
    validation_data=valid_gen_flow,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1, 
    epochs=epochs)

## Model Evaluation

In [ ]:
y_pred = model.predict(test_gen, verbose = 0)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(test_gen.classes, y_pred)
plt.figure(figsize=(10, 5))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", xticklabels=valid_gen_flow.class_indices, yticklabels=valid_gen_flow.class_indices)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
loss,accuracy = model.evaluate(test_gen, verbose = 0)
print(f"The accuarcy for this model is {accuracy:.0%}")

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
img1 = get_image("test1.jpg")
img2 = process(img1)
classes = np.array(list(train_gen_flow.class_indices))
y_pred = classes[np.argmax(model.predict(img2, verbose = 0), axis=1)]
plt.figure(figsize=(15,8))
plt.imshow(img1)
plt.title(y_pred[0])
plt.show()  



In [ ]:
model.save("meow")